# 03. Preprocessing Street View Housing Numbers (SVHN) Dataset

### Purpose:
Using the provided RBNR annotations, crop out the defined bibs and feed each bib into the digit detector.  During the cropping process, a text file containing the image names of the cropped bib files along their true RBN will be created.  A similar list will also be created for the predicted RBNs during the digit detection step.  These lists can then be compared in the validation section.

Set1 and Set2 of the RBNR dataset will be used later to train the bib detection model, but neither set has been used in training the digit detection model.  Therefore all three sets are being used as validation for this step.

### Before Running Notebook:
1. Create a folder named Validation under the top level of this repo.
1. Create a folder named Bibs under ./Data/Validation/.  This is where the croped bib images will be saved along with the list of image names and bib numbers in a text file for validation.
1. Download the config file and weights file from Google Drive for the digit detection model.  These were created in the previous notebook (02_SVHN_YOLOv4_tiny_Darknet_Roboflow.ipynb).  Save them in ./Data/YOLO/num_reader/ from the top level of this repo, and make sure they are set as the value of configPath and weightsPath in the Digit Detection section. 
1. Create a folder named Nums under ./Data/Validation/.  This is where the annotated images along with the text file containing the predicted bib numbers will be saved.

In [2]:
# imports
import cv2 as cv
import numpy as np
import scipy.io as sio
import os
import pandas as pd

# Crop Bibs
---

In [2]:
def get_cropped_bib(image, input_path, out_path):  
    """
    Read in the RBNR image bounding box information and use it to save
    cropped out images of the bibs in the original image.  Then write
    the cropped bib image name and RBN to file.
    
    Args
        image (str): name of original image
        input_path (str): path to directory of image
        out_path (str): directory where results are saved
        
    Returns
        None
    """
    
    #load image
    img = cv.imread(input_path + image)
    
    # load annotation file
    f = sio.loadmat(input_path + image + '.mat')

    #get bounding boxes and bib numbers
    boxes = f['tagp']
    numbers = f['number'].flatten()

    for i, box in enumerate(boxes):
        #convert box values to int
        (t, b, l, r) = [int(i) for i in box]
        # crop image and save
        crop_img = img[t:b, l:r]
        crop_name = image[:-4]+'_'+'bib_'+str(i+1)+'.JPG'
        cv.imwrite(out_path + crop_name, crop_img)
        # write race bib number to file
        rbn_file = open(output_path + 'bib_numbers.txt', 'a')
        rbn_file.writelines(f"{crop_name},{numbers[i]}\n")

In [4]:
# set input and output info for set1
images_path = '../Data/RBNR/set1_org/'
images = [file for file in os.listdir(images_path) if file[-3:]=='JPG']

output_path = '../Data/Validation/Bibs/'

In [5]:
#check for existing bib_numbers.txt and remove if exists
if os.path.exists(output_path + 'bib_numbers.txt'):
    os.remove(output_path + 'bib_numbers.txt')

In [6]:
for image in images:
    get_cropped_bib(image, images_path, output_path)

In [7]:
# repeat process for set2
images_path = '../Data/RBNR/set2_org/'
images = [file for file in os.listdir(images_path) if file[-3:]=='JPG']

for image in images:
    get_cropped_bib(image, images_path, output_path)

In [8]:
# repeat process for set3
images_path = '../Data/RBNR/set3_org/'
images = [file for file in os.listdir(images_path) if file[-3:]=='JPG']

for image in images:
    get_cropped_bib(image, images_path, output_path)

# Digit Detection
---

In [5]:
# get random colors for boxes
np.random.seed(42)
colors = np.random.randint(0, 255, size=(10, 3), dtype='uint8')

In [3]:
# Give the configuration and weight files for the model and load the network.
configPath = '../Data/YOLO/num_reader/SVHN3_custom-yolov4-tiny-detector.cfg'
weightsPath = '../Data/Yolo/num_reader/SVHN3_custom-yolov4-tiny-detector_best.weights'

net = cv.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

# determine the output layer
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [15]:
def create_labeled_image(image, input_path, out_path): 
    """
    Run digit detection and save a labeled image.  Then compile digits
    into single RBN and save to file for validation.
    Code for using YOLO in OpenCV adapted from OpenCV Docs:
    https://opencv-tutorial.readthedocs.io/en/latest/yolo/yolo.html
    
    Args
        image (str): name of original image
        input_path (str): path to directory of image
        out_path (str): directory where results are saved
        
    Returns
        None
    """
    # read in image and construct a blob from the image
    img = cv.imread(input_path + image)
    blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

    # get detections
    net.setInput(blob)
    outputs = net.forward(ln)

    # initialize lists
    boxes = []
    confidences = []
    classIDs = []
    
    # initialize image dimensions
    h_img, w_img = img.shape[:2]

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Only keep detection if it is for a digit with high confidence
            if confidence > 0.5:
                box = detection[:4] * np.array([w_img, h_img, w_img, h_img])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                box = [x, y, int(width), int(height)]
                boxes.append(box)
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    # get indices of final bounding boxes  
    indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    # initialize list for digit position and value
    bib_digit_loc = []
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            
            cv.rectangle(img, (x, y), (x + w, y + h), color, 1)
            text = "{}: {:.4f}".format(classIDs[i], confidences[i])
            cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            bib_digit_loc.append((x, str(classIDs[i])))
        
        # save annotated image
        cv.imwrite(out_path+image[:-4]+'_'+'detected'+'.JPG', img)
        
        # write race bib number to file
        bib_digit_loc.sort()
        rbn_pred = int(''.join([i[1] for i in bib_digit_loc]))
        #orig_image = '_'.join(image.split('_')[:2]) + '.JPG'
        rbn_pred_file = open(out_path + 'rbn_preds.txt', 'a')
        rbn_pred_file.writelines(f"{image},{rbn_pred}\n")

In [16]:
# set input and output info for detections
images_path = '../Data/Validation/Bibs/'
images = [file for file in os.listdir(images_path) if file[-3:]=='JPG']

output_path = '../Data/Validation/Nums/'

In [17]:
#check for existing bib_numbers.txt and remove if exists
if os.path.exists(output_path + 'rbn_preds.txt'):
    os.remove(output_path + 'rbn_preds.txt')

In [18]:
# run detections on all images in input directory
for image in images:
    create_labeled_image(image, images_path, output_path)

# Validation
---

## Training Validation
![Digit Detection](../Presentation/Images/SVHN_training_validation.png "Digit Detection")

In [3]:
true_df = pd.read_csv('../Data/Validation/Bibs/bib_numbers.txt', delimiter=',', 
                      index_col=0, names=['image', 'rbn'])
true_df.head()

,rbn
image,
set1_62_bib_1.JPG,941
set1_76_bib_1.JPG,3621
set1_89_bib_1.JPG,1703
set1_88_bib_1.JPG,1442
set1_77_bib_1.JPG,847


In [4]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, set1_62_bib_1.JPG to set3_42_bib_1.JPG
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   rbn     290 non-null    int64
dtypes: int64(1)
memory usage: 4.5+ KB


In [5]:
pred_df = pd.read_csv('../Data/Validation/Nums/rbn_preds.txt', delimiter=',', 
                      index_col=0, names=['image', 'pred_rbn'])
pred_df.head()

,pred_rbn
image,
set3_21_bib_2.JPG,3054
set1_29_bib_1.JPG,130
set2_27_bib_2.JPG,20927
set2_50_bib_1.JPG,89
set3_56_bib_1.JPG,2244


In [6]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, set3_21_bib_2.JPG to set3_03_bib_6.JPG
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   pred_rbn  286 non-null    int64
dtypes: int64(1)
memory usage: 4.5+ KB


In [7]:
all_df = pd.merge(true_df, pred_df, on='image', how='left')

In [8]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, set1_62_bib_1.JPG to set3_42_bib_1.JPG
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rbn       290 non-null    int64  
 1   pred_rbn  286 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.8+ KB


#### Accurate Predictions

In [9]:
all_df.loc[all_df['rbn'] == all_df['pred_rbn']]

,rbn,pred_rbn
image,,
set1_76_bib_1.JPG,3621,3621.0
set1_75_bib_1.JPG,1676,1676.0
set1_61_bib_1.JPG,1679,1679.0
set1_48_bib_1.JPG,663,663.0
set1_60_bib_1.JPG,1404,1404.0
...,...,...
set3_55_bib_2.JPG,4624,4624.0
set3_43_bib_1.JPG,2074,2074.0
set3_57_bib_2.JPG,4183,4183.0


#### No Prediction

In [11]:
all_df.loc[all_df['pred_rbn'].isna()]

,rbn,pred_rbn
image,,
set1_28_bib_1.JPG,311,NaN
set1_16_bib_1.JPG,1463,NaN
set1_17_bib_1.JPG,1463,NaN
set1_07_bib_1.JPG,979,NaN


#### Accuracy

In [14]:
true_positives = len(all_df.loc[all_df['rbn'] == all_df['pred_rbn']])
total = len(true_df)

true_positives / total

0.6758620689655173

# Conclusions
---

The overall accuracy of the digit detector when putting together RBN's could be higher; however, when looking at the inaccurate predictions, many are only off by a single digit.  Given that this model will be used in real time where an athlete can be moved to aquire the best possible read, it is most likely sufficient.  Also it has been shown by previous researchers that using the SVHN dataset in combination with a large set of racing bib number images for training produces a better result.  Further information on that study can be found [here](https://www.researchgate.net/publication/335234017_Racing_Bib_Number_Recognition_Using_Deep_Learning).  Future work will enclude gathering images more specifically related to the end goal of identifying a single bib number after the conclusion of the race, and retraining the model with that set included.